# CNMa
###### Correlation N Matching

In [1]:
#import IP_address_filter
#import SG_IP_matching
#import xml_to_csv
import VT_API
import OS_API
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os
from tqdm import tqdm
import json
import re

## Search in OpenSearch

In [23]:
#columns of interest
COLS = ['peerIP',
        'peerPort',
        'hostIP',
        'hostPort',
        'commands',
        'hashes',
        'urls',
        'loggedin',
        'startTime',
        'endTime',
        'sort_num',
        'peerCountry',
        'hostCountry'
       ]

SIZE = 10000

### How to craft filter query
Query inputting: list of dictionaries/jsons\
4 main filters:
- range: usually to filter on time ranges. time can be exact (2022-01-05T23:59:30) or relative (now-1w).
- exists: filter out records that have an existing field of choice
- match: filter out records that exactly matches a value on the chosen field, must match Exactly.
- query_string: filter out records that contains value(s) on the chosen field, can take in arguments such as AND and/or OR.

All filters are optional and can be repeated multiple times if require multiple matches. 

In [44]:
defaultquery = [{"range": {
                "startTime": {
                  "gte": "now-6w",
                  "lte": "now-4w",
                  "format": "strict_date_optional_time"
                }}},
                {"exists": {"field": "hashes"}},
                {"match": {"geoip.country_name": "Singapore"}},
                {"query_string": {"default_field": "commands", "query": "wget"}}
               ]

In [45]:
size = 10000 #batch size, max 10000
search_after = [0]
df_os = pd.DataFrame(columns=COLS)

queryls = defaultquery
print(queryls)

total = 0

while (size == SIZE):
    print(f"Continue from record: {search_after}")
    data = OS_API.opensearch_request(size,queryls,search_after)
    df_os = pd.concat([df_os,data])
    
    #update params
    size = len(data) #exit once it completes last batch
    total+=size #update total records number
    search_after = data['sort_num'][len(data)-1] #update sort number to continue after
    #time.sleep(1)
    
print(f'Done. Total records: {total}')

[{'range': {'startTime': {'gte': 'now-6w', 'lte': 'now-4w', 'format': 'strict_date_optional_time'}}}, {'exists': {'field': 'hashes'}}, {'match': {'geoip.country_name': 'Singapore'}}, {'query_string': {'default_field': 'commands', 'query': 'wget'}}]
Continue from record: [0]
<Response [200]>
Output success
Done. Total records: 20


In [46]:
df_os

,peerIP,peerPort,hostIP,hostPort,commands,hashes,urls,loggedin,startTime,endTime,sort_num,peerCountry,hostCountry
0,207.148.117.74,59868,5.8.16.12,22,[ln -sf /usr/sbin/sshd /tmp/su;/tmp/su -oPort=...,[f8c28666f2f2beb599dcc62721c41a82f52e63721dd2d...,[],"[root, admin]",2022-09-03T17:04:49.016421Z,2022-09-03T17:05:37.812201Z,[1662224737916],SG,RU
1,207.148.117.74,63707,103.159.132.105,22,[ln -sf /usr/sbin/sshd /tmp/su;/tmp/su -oPort=...,[f8c28666f2f2beb599dcc62721c41a82f52e63721dd2d...,[],"[root, admin]",2022-09-03T17:34:44.140172Z,2022-09-03T17:35:33.743336Z,[1662226533914],SG,NaN
2,45.64.130.149,47174,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, swifix!@#]",2022-09-08T13:43:24.211533Z,2022-09-08T13:44:26.916917Z,[1662644666983],SG,JP
3,45.64.130.149,45722,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, adminsato]",2022-09-08T13:43:53.371695Z,2022-09-08T13:44:56.217601Z,[1662644696282],SG,JP
4,45.64.130.149,43400,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, dd0s1n6]",2022-09-08T13:44:23.154376Z,2022-09-08T13:45:25.995672Z,[1662644726094],SG,JP
5,45.64.130.149,40838,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, born1981]",2022-09-08T13:44:53.365094Z,2022-09-08T13:45:56.036389Z,[1662644756201],SG,JP
6,45.64.130.149,38290,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, vH6BWp6Rtfm]",2022-09-08T13:45:25.043601Z,2022-09-08T13:46:27.803377Z,[1662644787868],SG,JP
7,45.64.130.149,34264,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, YE2$jAkE]",2022-09-08T13:45:58.697752Z,2022-09-08T13:47:01.393406Z,[1662644821469],SG,JP
8,45.64.130.149,60430,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, He+347-cPZ]",2022-09-08T13:46:33.580117Z,2022-09-08T13:47:36.510154Z,[1662644856634],SG,JP
9,45.64.130.149,57552,108.61.187.249,22,[uname -a;id;cat /etc/shadow /etc/passwd;lscpu...,[f494991106ba8bcca0641b6cc94e5423d5479ac87ef27...,"[http://sos.vivi.sg/ns1.jpg, http://sos.vivi.s...","[root, KVE6DSYLXMinIK]",2022-09-08T13:47:09.407707Z,2022-09-08T13:48:12.102680Z,[1662644892202],SG,JP


In [7]:
#export to csv
'''
df_os.to_csv("OS_pull" + "_" + str(datetime.now().strftime('%Y_%m_%d_%H_%M_%S')) + ".csv", index=False)
'''

'\ndf_os.to_csv("OS_pull" + "_" + str(datetime.now().strftime(\'%Y_%m_%d_%H_%M_%S\')) + ".csv", index=False)\n'

## Searching Hashes 

In [48]:
hashls = []

for i in df_os['hashes']:
    for j in i:
        if j not in hashls:
            hashls.append(j)

hashls

['f8c28666f2f2beb599dcc62721c41a82f52e63721dd2d5629073033b32a93154',
 'f494991106ba8bcca0641b6cc94e5423d5479ac87ef27d9dba5e9b827acd7212',
 'ae9e5007de8b71cca7a456d1913be332258c60b1fb1a444e8144d99251a144c7']

In [47]:
resultls = []

for hsh in tqdm(hashls):
    result = VT_API.virustotal_request(hsh)
    resultls.append(result)
    time.sleep(5)

resultls
    

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:18<00:00,  6.19s/it]


[{'query_input': 'f494991106ba8bcca0641b6cc94e5423d5479ac87ef27d9dba5e9b827acd7212',
  'stats_output': 0.6271186440677966,
  'engines_flagged': ['MicroWorld-eScan',
   'ClamAV',
   'FireEye',
   'CAT-QuickHeal',
   'ALYac',
   'Sangfor',
   'K7AntiVirus',
   'K7GW',
   'Arcabit',
   'Baidu',
   'Cyren',
   'Symantec',
   'ESET-NOD32',
   'TrendMicro-HouseCall',
   'Avast',
   'Cynet',
   'Kaspersky',
   'BitDefender',
   'NANO-Antivirus',
   'Ad-Aware',
   'Emsisoft',
   'DrWeb',
   'TrendMicro',
   'McAfee-GW-Edition',
   'Sophos',
   'Ikarus',
   'Avira',
   'Microsoft',
   'GData',
   'AhnLab-V3',
   'McAfee',
   'Rising',
   'Yandex',
   'MAX',
   'Fortinet',
   'AVG',
   'Panda'],
  'harmless': 0,
  'malicious': 37,
  'suspicious': 0,
  'undetected': 22,
  'timeout': 0},
 {'query_input': 'ae9e5007de8b71cca7a456d1913be332258c60b1fb1a444e8144d99251a144c7',
  'stats_output': 0.017543859649122806,
  'engines_flagged': ['DrWeb'],
  'harmless': 0,
  'malicious': 1,
  'suspicious': 0,
  

## Searching for urls within Commands

In [49]:
commandls = []

for i,row in df_os.iterrows():
    for c in row['commands']:
        try:
            #regex to find url starting with http(s)
            found = re.findall("https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[/]*)", c)
            #print(i, found)

            for f in found:
                if f not in commandls:
                    commandls.append(f)

        except AttributeError:
            print(i, 'No matches found')
            found = ''

commandls

['http://mdb7.cn', 'http://167.179.75.38', 'http://sos.vivi.sg/']

In [ ]:
#testing
'''
listed=['http://123.34.123.341|ersfgfadsilu  http://123.34.123.341|ersfgfadsilu', 'http://123.34.123.341', 'aiodusnfavic 123.34.123.341|ersfgfadsilu', 'http://123.34.123.341|ersfgfadsilu 123.34.123.341', 'vnoaids.abfna']
for c in listed:
    found = re.findall("https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)|[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)", c)
    print(found)
'''

In [51]:
commandresults = []

for com in tqdm(commandls):
    result = VT_API.virustotal_request(com)
    commandresults.append(result)
    time.sleep(5)
    
commandresults

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:17<00:00,  5.95s/it]


[{'query_input': 'http://mdb7.cn',
  'stats_output': 0.0,
  'engines_flagged': [],
  'harmless': 69,
  'malicious': 0,
  'suspicious': 0,
  'undetected': 11,
  'timeout': 0},
 {'query_input': 'http://167.179.75.38',
  'stats_output': 0.021505376344086023,
  'engines_flagged': ['Webroot', 'Kaspersky'],
  'harmless': 81,
  'malicious': 2,
  'suspicious': 0,
  'undetected': 10,
  'timeout': 0},
 {'query_input': 'http://sos.vivi.sg/',
  'stats_output': 0.011363636363636364,
  'engines_flagged': ['CRDF'],
  'harmless': 77,
  'malicious': 1,
  'suspicious': 0,
  'undetected': 10,
  'timeout': 0}]

## Work in Progress (Ignore below)
Interactive component to customise input

In [3]:
def yesnocheck(param):
    '''
    Forces input to be y or n, otherwise loops back.
    Returns y or n.
    '''
    while True:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        ans = input(f"{param}")
        if ans not in ['y', 'n']:
            continue            
        else:
            break
    return ans

#ans = yesnocheck("test")
#ans

In [ ]:
def craft_query(defmode):
    '''
    Interactive component to customize input
    '''
    #defmode to return default eg.
    while defmode:
        return defaultquery
    
    queryls = []
    
    #custom input
    #Time range
    ans = yesnocheck("Filter on Time Range? y/n")
    if ans == 'y':
        start = str(input("Input start date:"))
        end = str(input("Input end date:"))
        rnge = {"range": {"startTime": {"gte": start, "lte": end, "format": "strict_date_optional_time"}}}
        queryls.append(rnge)    

    #Existing field
    while True:
        ans = yesnocheck("Filter if a field Exists? y/n")
        #print(ans)
        if ans == 'y':
            exist = input("Please enter field:")
            exst = {'exists': {'field': exist}}
            queryls.append(exst)
        else:
            break
    
    #Exact match on field
    while True:
        ans = yesnocheck("Exact match on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            value = input("Enter value to match:")
            mtch = {"match": {field: value}}
            queryls.append(mtch)
        else:
            break
    
    #General match query
    while True:
        ans = yesnocheck("General search query on a field? y/n")
        #print(ans)
        if ans == 'y':
            field = input("Please enter field:")
            query = input("Enter query:")
            qry = {"query_string": {"default_field": field, "query": query}}
            queryls.append(qry)
        else:
            break
    
    return queryls
    
#queryls = craft_query(True)
#queryls